In [42]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [43]:
# Importing the dataset
dataset = pd.read_csv(r'^NSEI\1m till 2022-02-18.csv')
dataset

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-02-10 09:15:00+05:30,17554.10,17554.10,17512.15,17518.80,17518.80,0
1,2022-02-10 09:16:00+05:30,17517.15,17536.05,17516.50,17534.60,17534.60,0
2,2022-02-10 09:17:00+05:30,17534.65,17534.65,17522.55,17522.60,17522.60,0
3,2022-02-10 09:18:00+05:30,17523.40,17523.40,17508.05,17515.10,17515.10,0
4,2022-02-10 09:19:00+05:30,17514.25,17516.20,17500.75,17500.75,17500.75,0
...,...,...,...,...,...,...,...
2620,2022-02-18 15:26:00+05:30,17276.85,17277.25,17273.40,17274.70,17274.70,0
2621,2022-02-18 15:27:00+05:30,17273.85,17276.25,17272.10,17274.45,17274.45,0
2622,2022-02-18 15:28:00+05:30,17274.60,17274.65,17263.10,17266.00,17266.00,0
2623,2022-02-18 15:29:00+05:30,17265.95,17271.30,17263.65,17268.75,17268.75,0


In [44]:
dataset.drop(['Datetime', 'Adj Close', 'Volume'], axis=1, inplace=True)
dataset

,Open,High,Low,Close
0,17554.10,17554.10,17512.15,17518.80
1,17517.15,17536.05,17516.50,17534.60
2,17534.65,17534.65,17522.55,17522.60
3,17523.40,17523.40,17508.05,17515.10
4,17514.25,17516.20,17500.75,17500.75
...,...,...,...,...
2620,17276.85,17277.25,17273.40,17274.70
2621,17273.85,17276.25,17272.10,17274.45
2622,17274.60,17274.65,17263.10,17266.00
2623,17265.95,17271.30,17263.65,17268.75


In [45]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
dataset_scaled = sc.fit_transform(dataset)
dataset_scaled

array([[0.89779608, 0.89690659, 0.85155539, 0.8549743 ],
       [0.85305159, 0.87505297, 0.856831  , 0.87408527],
       [0.87424316, 0.87335795, 0.86416833, 0.85957061],
       ...,
       [0.5593364 , 0.55856892, 0.54951185, 0.54919867],
       [0.54886171, 0.55451299, 0.55017889, 0.55252495],
       [0.56139501, 0.56056662, 0.56552059, 0.56165709]])

In [46]:
# Creating a data structure with 60 timesteps and 1 output
x_train = []
y_train = []
for i in range(60, int(len(dataset_scaled)*0.8)):
    x_train.append(dataset_scaled[i-60:i])
    y_train.append(dataset_scaled[i, 3])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train.shape, y_train.shape

((2040, 60, 4), (2040,))

In [47]:
# Creating a data structure with 60 timesteps and 1 output
x_test = []
y_test = []
for i in range(int(len(dataset_scaled)*0.8), len(dataset_scaled)):
    x_test.append(dataset_scaled[i-60:i])
    y_test.append(dataset_scaled[i, 3])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test.shape, y_test.shape

((525, 60, 4), (525,))

In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(x_train.shape[0],))
x = layers.Dense(x_train.shape[1], activation="relu")(inputs)
x = layers.Dense(x_train.shape[2], activation="relu")(x)
model = keras.Model(inputs=inputs, outputs=x, name="mnist_model")
model.summary()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2040)]            0         
                                                                 
 dense_4 (Dense)             (None, 60)                122460    
                                                                 
 dense_5 (Dense)             (None, 4)                 244       
                                                                 
Total params: 122,704
Trainable params: 122,704
Non-trainable params: 0
_________________________________________________________________


In [49]:
#Saving the model
import pickle
classifier = open('classifier.model', 'wb')
pickle.dump(model, classifier)
classifier.close()

INFO:tensorflow:Assets written to: ram://5cb43ad4-4a3f-4964-a931-39e103aa865a/assets


In [50]:
#Loading saved model
classifier = open('classifier.model', 'rb')
model2 = pickle.load(classifier)
classifier.close()

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://43abf4de-4c89-473d-8ad4-cc55c6357bf4/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [ ]:
#Testing the model
score = model2.score(x_test[0], y_test[0])
print("Accuracy:", score)

ValueError: in user code:

    File "c:\Users\Genus\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\Genus\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Genus\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\Genus\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\Genus\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Genus\AppDa